In [1]:
# %pip install pandas
# %pip install numpy
# %pip install scikit-learn
# %pip install kagglehub

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import kagglehub
import cupy as cp


c:\Users\patry\anaconda3\envs\weather\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
historical_hourly_weather_data_path = kagglehub.dataset_download('selfishgene/historical-hourly-weather-data')

city = "Portland"

city_attributes = pd.read_csv(f"{historical_hourly_weather_data_path}/city_attributes.csv")
humidity = pd.read_csv(f"{historical_hourly_weather_data_path}/humidity.csv")
pressure = pd.read_csv(f"{historical_hourly_weather_data_path}/pressure.csv")
temperature = pd.read_csv(f"{historical_hourly_weather_data_path}/temperature.csv")
weather_description = pd.read_csv(f"{historical_hourly_weather_data_path}/weather_description.csv")
wind_speed = pd.read_csv(f"{historical_hourly_weather_data_path}/wind_speed.csv")
wind_direction = pd.read_csv(f"{historical_hourly_weather_data_path}/wind_direction.csv")

In [4]:
if city not in city_attributes['City'].values:
    raise ValueError(f"City '{city}' does not exist in the data. Available cities are: {city_attributes['City'].unique()}")

selected_city = city_attributes[city_attributes['City'] == city].index[0]
data_frames = [humidity, pressure, temperature, weather_description, wind_speed, wind_direction]

for i, df in enumerate(data_frames):
    df.set_index('datetime', inplace=True)
    data_frames[i] = df.iloc[:, selected_city]

In [5]:
combined_data = pd.concat(data_frames, axis=1)
combined_data.columns = [
    'humidity', 'pressure', 'temperature', 'weather_description', 
    'wind_speed', 'wind_direction'
]
combined_data.index = pd.to_datetime(combined_data.index)

# aggregate daily
aggregated_data = combined_data.resample('D').agg({
    'temperature': 'mean',
    'humidity': 'mean',
    'wind_speed': ['max', 'mean'],
    'pressure': 'mean',
    'weather_description': lambda x: x.mode()[0] if not x.mode().empty else np.nan,
    'wind_direction': 'mean'
})
aggregated_data.columns = ['_'.join(col).strip('_') for col in aggregated_data.columns.values]

In [6]:
# encoder = LabelEncoder()
# aggregated_data['weather_description'] = encoder.fit_transform(aggregated_data['weather_description'])
# weather_mapping = dict(enumerate(encoder.classes_))
# print("Mapping for weather_description:", weather_mapping)

In [7]:
def preprocess_weather_data(data, window_size=3):
    X, y = [], []
    for i in range(window_size, len(data) - 1):
        X_window = data.iloc[i-window_size:i][[
            'temperature_mean', 'humidity_mean', 'pressure_mean', 'wind_speed_max', 'wind_speed_mean', 'wind_direction_mean'
        ]].values
        y_target = data.iloc[i + 1][['temperature_mean', 'wind_speed_max']].values

        # encode wind speed >= 6 as binary
        y_target[1] = 1 if y_target[1] >= 6 else 0
        X.append(X_window)
        y.append(y_target)

    X = np.array(X)
    y = np.array(y)

    # normalize X
    X_mean = X.mean(axis=(0, 1), keepdims=True)
    X_std = X.std(axis=(0, 1), keepdims=True)
    X_normalized = (X - X_mean) / (X_std + 1e-9)

    return X_normalized, y

In [8]:
X, y = preprocess_weather_data(aggregated_data, window_size=3)

# split into train/test (0.7 or 0.8)
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [9]:
print(X_train.shape)
print(y_train.shape)

print(X_test.shape)
print(y_test.shape)

(1506, 3, 6)
(1506, 2)
(377, 3, 6)
(377, 2)


In [10]:
# print(X_train)
# print(y_train)

In [11]:
def train_and_evaluate(model, X_train, y_train, X_test, y_test, epochs=1000, learning_rate=0.001, bath_size=128):
    X_train_cp = cp.array(X_train.reshape(X_train.shape[0], -1), dtype=cp.float32)
    y_train_cp = cp.array(y_train, dtype=cp.float32)
    X_test_cp = cp.array(X_test.reshape(X_test.shape[0], -1), dtype=cp.float32)
    y_test_cp = cp.array(y_test, dtype=cp.float32)

    model.train(X_train_cp, y_train_cp, X_test_cp, y_test_cp, epochs, learning_rate, batch_size=bath_size)

    predictions = model.predict(X_test_cp)
    
    mae = cp.mean(cp.abs(predictions[:, 0] - y_test_cp[:, 0]))

    from sklearn.metrics import roc_auc_score
    auc = roc_auc_score(cp.asnumpy(y_test_cp[:, 1]), cp.asnumpy(predictions[:, 1]))

    print(f"Test Regression MAE: {mae}")
    print(f"Test Classification AUC: {auc}")

    print("Predictions:" , predictions[:5, :])
    print("True values:", y_test_cp[:5, :])

    return mae, auc

In [12]:
from weather_prediction import WeatherPredictionNetwork

layers = [X_train.shape[1] * X_train.shape[2], 128, 64, 2]
activations = ["relu", "relu"]
model = WeatherPredictionNetwork(layers, activations, seed=21)

train_and_evaluate(model, X_train, y_train, X_test, y_test, epochs=10000, learning_rate=0.01, bath_size=128)

Epoch 0, Regression Loss: 205.7767333984375, Classification AUC: 0.6998396519437997, Learning Rate: 0.001
Test Regression MAE: 218.5678253173828, Test Classification AUC: 0.5
Epoch 100, Regression Loss: 3.4849894046783447, Classification AUC: 0.6413361590473308, Learning Rate: 0.001
Epoch 200, Regression Loss: 3.1545827388763428, Classification AUC: 0.6782991892891985, Learning Rate: 0.001
Epoch 300, Regression Loss: 3.098140001296997, Classification AUC: 0.7042352070508292, Learning Rate: 0.001
Epoch 400, Regression Loss: 2.8918004035949707, Classification AUC: 0.7179825299110798, Learning Rate: 0.001
Epoch 500, Regression Loss: 2.9156625270843506, Classification AUC: 0.724977293369664, Learning Rate: 0.001
Epoch 600, Regression Loss: 3.137298345565796, Classification AUC: 0.7268319485092116, Learning Rate: 0.001
Epoch 700, Regression Loss: 2.8282148838043213, Classification AUC: 0.7309763290386966, Learning Rate: 0.001
Epoch 800, Regression Loss: 2.894335985183716, Classification AUC

(array(2.61858, dtype=float32), np.float64(0.5631442548482907))